In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from IPython.display import display, HTML

def b_print(df , n=30 , clean=True):
    
    # from IPython.display import display, HTML

    if clean : # remove tickers da mesma empresa, deixando a primeria ocorrencia
        df['prefixo'] = df['Papel'].astype(str).str[:4]
        df=df.drop_duplicates(subset='prefixo', keep='first')
        # df=df.drop('prefixo', axis=1) 
    
    display(HTML(df.head(n).to_html(index=False)))
    df = None


In [2]:
# A empresa e Indicadores fundamentalistas dados do statusinvest.com.br
#(pode ter atraso, dados não são pegos diretamente no SI)

url = 'https://raw.githubusercontent.com/BDonadelli/Codigos-em-financas/main/data/SI_Acoes.csv'
pd.set_option('display.max_columns', None)
funds = pd.read_csv(url,sep=';' , decimal=',' ,thousands ='.' )

In [3]:
funds.columns

Index(['TICKER', 'PRECO', 'DY', 'P/L', 'P/VP', 'P/ATIVOS', 'MARGEM BRUTA',
       'MARGEM EBIT', 'MARG. LIQUIDA', 'P/EBIT', 'EV/EBIT',
       'DIVIDA LIQUIDA / EBIT', 'DIV. LIQ. / PATRI.', 'PSR', 'P/CAP. GIRO',
       'P. AT CIR. LIQ.', 'LIQ. CORRENTE', 'ROE', 'ROA', 'ROIC',
       'PATRIMONIO / ATIVOS', 'PASSIVOS / ATIVOS', 'GIRO ATIVOS',
       'CAGR RECEITAS 5 ANOS', 'CAGR LUCROS 5 ANOS', ' LIQUIDEZ MEDIA DIARIA',
       ' VPA', ' LPA', ' PEG Ratio', ' VALOR DE MERCADO'],
      dtype='object')

In [4]:
fundsSI =  funds[funds[' LIQUIDEZ MEDIA DIARIA'] > 250000] 
fundsSI =  fundsSI[(fundsSI[' LPA'] > 0) & (fundsSI[' VPA'] > 0) ]
# fundsSI

In [5]:
fundsSI['valor_intrinseco'] = np.sqrt(22.5 * fundsSI[' LPA'] * fundsSI[' VPA'])
fundsSI['Delta'] = fundsSI['valor_intrinseco'] / fundsSI['PRECO'] -1
fundsSI["Rank"]   = fundsSI['Delta'].rank(ascending=True, method="min")
fundsSI.sort_values(by="Rank", ascending=False, inplace=True)
fundsSI.reset_index(inplace=True)
fundsSI.index = fundsSI.index + 1

top20 = fundsSI[['TICKER','PRECO' , 'valor_intrinseco' , 'Rank' , 'Delta' , ' LPA' , ' VPA', 'P/L' ]].head(20)
top20

,TICKER,PRECO,valor_intrinseco,Rank,Delta,LPA,VPA,P/L
1,LUPA3,1.45,8.681230,203.0,4.987055,0.87,3.85,1.67
2,MEAL3,1.51,5.982725,202.0,2.962070,0.41,3.88,3.70
3,MTRE3,3.93,13.056722,201.0,2.322321,0.82,9.24,4.81
4,BRSR6,11.10,34.915348,200.0,2.145527,2.29,23.66,4.84
5,HBOR3,2.67,8.386596,199.0,2.141047,0.30,10.42,8.83
6,VVEO3,3.52,10.440905,198.0,1.966166,0.50,9.69,7.02
7,HBRE3,5.00,14.570089,197.0,1.914018,0.51,18.50,9.78
8,GOAU3,11.08,32.252023,196.0,1.910832,2.66,17.38,4.17
9,GOAU4,11.11,32.252023,195.0,1.902972,2.66,17.38,4.18
10,EUCA4,14.02,39.888636,194.0,1.845124,2.97,23.81,4.72


In [6]:
fundsSI =  fundsSI[(fundsSI['P/L'] > 0)]
top20 = fundsSI[['TICKER','PRECO' , 'valor_intrinseco' , 'Rank' , 'Delta' , ' LPA' , ' VPA' ]].head(20)
top20

,TICKER,PRECO,valor_intrinseco,Rank,Delta,LPA,VPA
1,LUPA3,1.45,8.681230,203.0,4.987055,0.87,3.85
2,MEAL3,1.51,5.982725,202.0,2.962070,0.41,3.88
3,MTRE3,3.93,13.056722,201.0,2.322321,0.82,9.24
4,BRSR6,11.10,34.915348,200.0,2.145527,2.29,23.66
5,HBOR3,2.67,8.386596,199.0,2.141047,0.30,10.42
6,VVEO3,3.52,10.440905,198.0,1.966166,0.50,9.69
7,HBRE3,5.00,14.570089,197.0,1.914018,0.51,18.50
8,GOAU3,11.08,32.252023,196.0,1.910832,2.66,17.38
9,GOAU4,11.11,32.252023,195.0,1.902972,2.66,17.38
10,EUCA4,14.02,39.888636,194.0,1.845124,2.97,23.81


In [7]:
fundsSI =  fundsSI[(fundsSI['CAGR RECEITAS 5 ANOS'] > 0) ]
top20 = fundsSI[['TICKER','PRECO' , 'valor_intrinseco' , 'Rank' , 'Delta' , ' LPA' , ' VPA' ]].head(20)
top20

,TICKER,PRECO,valor_intrinseco,Rank,Delta,LPA,VPA
2,MEAL3,1.51,5.982725,202.0,2.962070,0.41,3.88
3,MTRE3,3.93,13.056722,201.0,2.322321,0.82,9.24
4,BRSR6,11.10,34.915348,200.0,2.145527,2.29,23.66
5,HBOR3,2.67,8.386596,199.0,2.141047,0.30,10.42
7,HBRE3,5.00,14.570089,197.0,1.914018,0.51,18.50
8,GOAU3,11.08,32.252023,196.0,1.910832,2.66,17.38
9,GOAU4,11.11,32.252023,195.0,1.902972,2.66,17.38
10,EUCA4,14.02,39.888636,194.0,1.845124,2.97,23.81
11,COCE5,34.15,92.970298,193.0,1.722410,7.05,54.49
12,GGBR3,17.05,45.655230,192.0,1.677726,3.86,24.00


In [8]:
fundsSI =  fundsSI[(fundsSI['CAGR LUCROS 5 ANOS'] > 0) ]
top20 = fundsSI[['TICKER','PRECO' , 'valor_intrinseco' , 'Rank' , 'Delta' , ' LPA' , ' VPA' ]].head(20)
top20

,TICKER,PRECO,valor_intrinseco,Rank,Delta,LPA,VPA
2,MEAL3,1.51,5.982725,202.0,2.962070,0.41,3.88
10,EUCA4,14.02,39.888636,194.0,1.845124,2.97,23.81
11,COCE5,34.15,92.970298,193.0,1.722410,7.05,54.49
13,ALOS3,21.20,56.101845,191.0,1.646313,5.46,25.62
16,NEOE3,19.20,48.354565,188.0,1.518467,4.28,24.28
19,BMGB4,3.24,7.791341,185.0,1.404735,0.38,7.10
22,ALLD3,7.43,17.398772,182.0,1.341692,0.81,16.61
24,CLSC4,72.00,166.000928,180.0,1.305568,14.82,82.64
25,NTCO3,17.25,39.493123,179.0,1.289456,3.43,20.21
26,SAPR3,5.20,11.827383,178.0,1.274497,0.99,6.28


In [9]:
fundsSI =  fundsSI[(fundsSI['DIVIDA LIQUIDA / EBIT'] <4) ]
top20 = fundsSI[['TICKER','PRECO' , 'valor_intrinseco' , 'Rank' , 'Delta' , ' LPA' , ' VPA' ]].head(20)
top20

,TICKER,PRECO,valor_intrinseco,Rank,Delta,LPA,VPA
2,MEAL3,1.51,5.982725,202.0,2.962070,0.41,3.88
10,EUCA4,14.02,39.888636,194.0,1.845124,2.97,23.81
13,ALOS3,21.20,56.101845,191.0,1.646313,5.46,25.62
16,NEOE3,19.20,48.354565,188.0,1.518467,4.28,24.28
22,ALLD3,7.43,17.398772,182.0,1.341692,0.81,16.61
24,CLSC4,72.00,166.000928,180.0,1.305568,14.82,82.64
25,NTCO3,17.25,39.493123,179.0,1.289456,3.43,20.21
26,SAPR3,5.20,11.827383,178.0,1.274497,0.99,6.28
29,SAPR11,27.35,59.036120,175.0,1.158542,4.93,31.42
31,SAPR4,5.54,11.827383,173.0,1.134907,0.99,6.28


In [10]:
fundsSI['prefixo'] = fundsSI['TICKER'].str[:4]
fundsSI.drop_duplicates(subset='prefixo', keep='first').head(20)
df_limpo = fundsSI.drop_duplicates(subset='prefixo', keep='first')
df_limpo = df_limpo.drop('prefixo', axis=1) 
df_limpo[['TICKER','PRECO' , 'valor_intrinseco' , 'Rank' , 'Delta' , ' LPA' , ' VPA' ]].head(20)

,TICKER,PRECO,valor_intrinseco,Rank,Delta,LPA,VPA
2,MEAL3,1.51,5.982725,202.0,2.962070,0.41,3.88
10,EUCA4,14.02,39.888636,194.0,1.845124,2.97,23.81
13,ALOS3,21.20,56.101845,191.0,1.646313,5.46,25.62
16,NEOE3,19.20,48.354565,188.0,1.518467,4.28,24.28
22,ALLD3,7.43,17.398772,182.0,1.341692,0.81,16.61
24,CLSC4,72.00,166.000928,180.0,1.305568,14.82,82.64
25,NTCO3,17.25,39.493123,179.0,1.289456,3.43,20.21
26,SAPR3,5.20,11.827383,178.0,1.274497,0.99,6.28
33,ROMI3,10.39,21.952950,171.0,1.112892,1.76,12.17
35,FESA4,8.01,16.778558,169.0,1.094701,1.36,9.20


In [11]:
# linhas_com_nan = top20[top20.isna().any(axis=1)]
# linhas_com_nan

### Porque 22,5?

O número 22,5 na Fórmula de Graham é um fator de ponderação que tem um propósito específico. Essa constante foi escolhida por Graham pra ajustar a avaliação do preço justo de uma ação com base na taxa de crescimento anual esperada da empresa.

O número 22,5 é, na verdade, o resultado da multiplicação de 8,5 por 2,65 (8,5 x 2,65 = 22,5). O número 8,5 é a base que Graham considerou razoável pra uma empresa com taxa de crescimento zero, ou seja, uma empresa que não cresce.

Já o número 2,65 representa a média do retorno exigido pelos investidores no mercado de ações durante a época de Graham, que era de aproximadamente 4,4% acima da taxa de retorno dos títulos do Tesouro dos Estados Unidos.

O fator 22,5 ajuda a ajustar o preço justo com base no crescimento da empresa e na expectativa de retorno dos investidores.

Esse ajuste garante que a Fórmula de Graham considere a taxa de crescimento anual esperada e reflita uma avaliação mais realista do preço justo de uma ação.

No próximo tópico, abordaremos a calculadora do preço justo usando a Fórmula de Graham.